In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from dotenv import load_dotenv
import torch
from transformers import pipeline

In [2]:
load_dotenv()  # carrega as variáveis do .env
path = Path(os.getenv("DATASETS_PATH"))

books = pd.read_csv(os.path.join(path, "books_with_categories.csv"))

In [3]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction


In [4]:
device=0 if torch.cuda.is_available() else -1  # -1 para CPU, 0 para GPU

from transformers import pipeline

classifier = pipeline(
    "text-classification",
    # model="j-hartmann/emotion-english-distilroberta-base",  
    model="bhadresh-savani/distilbert-base-uncased-emotion",
    top_k=None,
    device=0 # GPU
)

Device set to use cuda:0


In [5]:
# Teste do classificadora
classifier("I love this!")

[[{'label': 'joy', 'score': 0.640127956867218},
  {'label': 'anger', 'score': 0.24070216715335846},
  {'label': 'love', 'score': 0.06310220062732697},
  {'label': 'sadness', 'score': 0.04607994854450226},
  {'label': 'fear', 'score': 0.007437662687152624},
  {'label': 'surprise', 'score': 0.002550082979723811}]]

In [6]:
books["description"][0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [7]:
classifier(books["description"][0])

[[{'label': 'fear', 'score': 0.8530158996582031},
  {'label': 'sadness', 'score': 0.11590155959129333},
  {'label': 'surprise', 'score': 0.020371900871396065},
  {'label': 'joy', 'score': 0.004783167503774166},
  {'label': 'anger', 'score': 0.003332399995997548},
  {'label': 'love', 'score': 0.0025950961280614138}]]

In [8]:
# Na mesma descrição do livro classificador pode retornar várias emoções, com isso vamos fazer a separação por frases separadas por ponto final
classifier(books["description"][0].split("."))

[[{'label': 'joy', 'score': 0.8328845500946045},
  {'label': 'surprise', 'score': 0.1558757722377777},
  {'label': 'fear', 'score': 0.004632767289876938},
  {'label': 'sadness', 'score': 0.002546006115153432},
  {'label': 'anger', 'score': 0.0022666361182928085},
  {'label': 'love', 'score': 0.0017942286795005202}],
 [{'label': 'joy', 'score': 0.6221776008605957},
  {'label': 'anger', 'score': 0.28773224353790283},
  {'label': 'fear', 'score': 0.04725947603583336},
  {'label': 'sadness', 'score': 0.02662086673080921},
  {'label': 'love', 'score': 0.011507333256304264},
  {'label': 'surprise', 'score': 0.004702533129602671}],
 [{'label': 'joy', 'score': 0.9378705024719238},
  {'label': 'sadness', 'score': 0.031638287007808685},
  {'label': 'anger', 'score': 0.010197604075074196},
  {'label': 'fear', 'score': 0.009079602546989918},
  {'label': 'surprise', 'score': 0.008866822347044945},
  {'label': 'love', 'score': 0.0023470944724977016}],
 [{'label': 'fear', 'score': 0.7420214414596558}

- Verifica se ao fazer o split ele continua fazendo a análise corretamente

In [9]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [10]:
sentences[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives'

In [11]:
predictions[0]

[{'label': 'joy', 'score': 0.8328845500946045},
 {'label': 'surprise', 'score': 0.1558757722377777},
 {'label': 'fear', 'score': 0.004632767289876938},
 {'label': 'sadness', 'score': 0.002546006115153432},
 {'label': 'anger', 'score': 0.0022666361182928085},
 {'label': 'love', 'score': 0.0017942286795005202}]

In [12]:
sentences[3]

' Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist'

In [13]:
predictions[3]

[{'label': 'fear', 'score': 0.7420214414596558},
 {'label': 'anger', 'score': 0.24075813591480255},
 {'label': 'sadness', 'score': 0.011440446600317955},
 {'label': 'surprise', 'score': 0.0023504579439759254},
 {'label': 'joy', 'score': 0.0020884631667286158},
 {'label': 'love', 'score': 0.0013410841347649693}]

- Com a nova estrategia podemos criar uma coluna com cada uma dessas categorias e retornar o maior valor possível dela para cada uma das emoções classificadas

In [14]:
emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_scores(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    return {label: np.max(scores) for label, scores in per_emotion_scores.items()}

In [15]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_scores(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])

100%|██████████| 5197/5197 [02:11<00:00, 39.48it/s]


In [16]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn

In [17]:
emotions_df

,anger,disgust,fear,joy,sadness,surprise,isbn13
0,0.473060,0.742021,0.998075,0.020045,0.998452,0.155876,9780002005883
1,0.705848,0.985500,0.995067,0.025595,0.110962,0.029527,9780002261982
2,0.473060,0.174097,0.989455,0.020045,0.110962,0.018560,9780006178736
3,0.478481,0.174097,0.962997,0.932864,0.110962,0.018560,9780006280897
4,0.794046,0.401118,0.995202,0.020045,0.110962,0.018560,9780006280934
...,...,...,...,...,...,...,...
5192,0.996865,0.298724,0.998706,0.014562,0.466609,0.384034,9788172235222
5193,0.736218,0.174097,0.994803,0.020045,0.110962,0.018560,9788173031014
5194,0.043854,0.268792,0.998804,0.004777,0.105466,0.025938,9788179921623
5195,0.911210,0.988397,0.998378,0.020045,0.997983,0.018560,9788185300535


In [18]:
# Fazendo merge com o dataframe original
books = pd.merge(books, emotions_df, on = "isbn13")
books.head()

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories,anger,disgust,fear,joy,sadness,surprise
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction,0.473060,0.742021,0.998075,0.020045,0.998452,0.155876
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction,0.705848,0.985500,0.995067,0.025595,0.110962,0.029527
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction,0.473060,0.174097,0.989455,0.020045,0.110962,0.018560
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction,0.478481,0.174097,0.962997,0.932864,0.110962,0.018560
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction,0.794046,0.401118,0.995202,0.020045,0.110962,0.018560


In [19]:
# Salvando a base de dados com as categorias

# Drop colunas que não serão utilizadas e salvas o arquivos em .csv
load_dotenv()  # carrega as variáveis do .env

# pega o caminho da variável de ambiente e converte em Path
path = Path(os.getenv("DATASETS_PATH"))
path.mkdir(parents=True, exist_ok=True)  # cria a pasta se não existir

(
    books
    .to_csv(f"{path}/books_with_emotions.csv", index=False)
)